In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import train as train

import loss as loss
import unet as unet
import random
import paths as paths
import numpy as np
import glob

# The meaning of life
random.seed(42)  
np.random.seed(42)

In [ ]:
DATA_MEAN = 168.3172158554484
DATA_STD = 340.21625683608994
BATCH_SIZE = 3
OUTPUT_CHANNELS = 1
DATA_PATH = "./vet_dataset_cleaned/"

In [ ]:
#WEIGHTS = "./vacbag/dsc_after_bce_epoch_" + "80" + ".hdf5"
WEIGHTS = "./vacbag_2/bce_epoch_68.hdf5"
INITIAL_LR = 1e-6
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = loss.dsc_loss
METRICS = [loss.dice_metric, 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall()]

model = unet.model(output_channels=OUTPUT_CHANNELS)
model.compile(OPTIMIZER, LOSS, METRICS)
model.load_weights(WEIGHTS)

In [ ]:
patient_paths = paths.get_patient_paths(DATA_PATH)
patient_paths.sort()

img_paths = [glob.glob(path + "/img/*") for path in patient_paths]
mask_paths = [glob.glob(path + "/mask/*") for path in patient_paths]

valid = int(len(img_paths) * 0.15 // 1)
test = int(len(img_paths) * 0.1 // 1)
train = int(len(img_paths) - valid - test)

train_inputs = paths.flatten_list(img_paths[0:train])
train_truths = paths.flatten_list(mask_paths[0:train])

train_inputs.sort()
train_truths.sort()

valid_inputs = paths.flatten_list(img_paths[train:train+valid])
valid_truths = paths.flatten_list(mask_paths[train:train+valid])

valid_inputs.sort()
valid_truths.sort()

test_inputs = paths.flatten_list(img_paths[train+valid:])
test_truths = paths.flatten_list(mask_paths[train+valid:])

test_inputs.sort()
test_truths.sort()

In [ ]:
test_inputs = np.array([np.load(array) for array in test_inputs])
test_truths = np.array([np.load(array) for array in test_truths])

In [ ]:
test_inputs = (test_inputs - DATA_MEAN) / DATA_STD

In [ ]:
test_inputs.shape, test_truths.shape

In [ ]:
model.evaluate(x=test_inputs, y=test_truths, batch_size=3)

In [ ]:
import explore as explore

In [ ]:
explore.plot_batch_predict(model, test_inputs, test_truths, x=7, batch_size = 3)

In [ ]:
from matplotlib import pyplot as plt

for index in range(len(test_inputs)):
    img = test_inputs[index:index+1]
    truth = test_truths[index:index+1]
    pred = model.predict(img)
    pred = np.round(pred)

    plt.figure(figsize=(5,5))
    plt.imshow(img[0,...,0], cmap='gray')
    plt.contour(truth[0,...,0], colors='yellow', alpha=0.5)
    plt.contour(pred[0,...,0], colors='red', alpha=0.5)
    plt.show()